In [2]:
import pandas as pd
df = pd.read_csv("/home/mac/project_ws/eda-repo-5/data/날씨_관련_데이터/자치구단위 서울생활인구 일별 집계표.csv", encoding= "euc-kr")

In [3]:
import mysql.connector

def execute_sql(sql, params=None):
    ojk = mysql.connector.connect(
        host = "database-1.cnu82kme6p4d.ap-northeast-2.rds.amazonaws.com",
        port = 3306,
        user = "eda",
        password = "ojk0707",
        database = "ojk"
    )
    cursor = ojk.cursor()

    try:
        if params:
            cursor.execute(sql, params)
        else:
            cursor.execute(sql)
        
        ojk.commit()
    except Exception as e:
        print(f"Error: {e}")
        ojk.rollback()
    finally:
        cursor.close()
        ojk.close()

In [4]:
import mysql.connector

def select_sql(sql):
    ojk = mysql.connector.connect(
        host = "database-1.cnu82kme6p4d.ap-northeast-2.rds.amazonaws.com",
        port = 3306,
        user = "eda",
        password = "ojk0707",
        database = "ojk"
    )
    cursor = ojk.cursor()

    cursor.execute(sql)

    results = cursor.fetchall()  # 모든 결과를 읽습니다
    ojk.close()
    cursor.close()
    return results

In [5]:
df

,기준일ID,시군구코드,시군구명,총생활인구수,내국인생활인구수,장기체류외국인인구수,단기체류외국인인구수,일최대인구수,일최소인구수,주간인구수(09~18),야간인구수(19~08),일최대이동인구수,서울외유입인구수,동일자치구행정동간이동인구수,자치구간이동인구수
0,20250327,11000,서울시,1.093891e+07,1.039568e+07,377012.2644,166214.2461,1.135267e+07,1.053806e+07,1.127657e+07,1.069772e+07,5.964684e+06,1.619183e+06,1.829103e+06,2.516397e+06
1,20250327,11110,종로구,3.359373e+05,3.025490e+05,16690.6847,16697.5710,4.560699e+05,2.328928e+05,4.301549e+05,2.686390e+05,2.755915e+05,7.955425e+04,2.639568e+04,1.696416e+05
2,20250327,11140,중구,3.571376e+05,2.974964e+05,19134.2024,40507.0099,5.157442e+05,2.195563e+05,4.825689e+05,2.675439e+05,3.356017e+05,1.011784e+05,2.467516e+04,2.097482e+05
3,20250327,11170,용산구,2.955388e+05,2.730957e+05,14027.1896,8415.9641,3.319861e+05,2.552596e+05,3.253080e+05,2.742752e+05,1.854691e+05,5.673884e+04,3.587798e+04,9.285225e+04
4,20250327,11200,성동구,3.679149e+05,3.494064e+05,14642.6658,3865.8410,3.955154e+05,3.433482e+05,3.887017e+05,3.530671e+05,2.131262e+05,5.040232e+04,5.009943e+04,1.126245e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65905,20180405,11620,관악구,5.272641e+05,5.036422e+05,22109.7135,1512.2282,5.954737e+05,4.520649e+05,4.662984e+05,5.708111e+05,2.526015e+05,3.352409e+04,1.443650e+05,7.471245e+04
65906,20180405,11650,서초구,6.570409e+05,6.366557e+05,14811.8716,5573.3269,7.830256e+05,5.230630e+05,7.620019e+05,5.820688e+05,5.133245e+05,1.595478e+05,9.886215e+04,2.549145e+05
65907,20180405,11680,강남구,9.128639e+05,8.713715e+05,22101.3558,19390.9906,1.147911e+06,6.928787e+05,1.105394e+06,7.753422e+05,7.517665e+05,2.261431e+05,1.402224e+05,3.854011e+05
65908,20180405,11710,송파구,7.733757e+05,7.531781e+05,14761.5934,5436.0700,7.892861e+05,7.626648e+05,7.703412e+05,7.755432e+05,4.373945e+05,1.140617e+05,1.853804e+05,1.379524e+05


### 2024년 기준 인구

In [6]:
group_df = df[['기준일ID',"시군구명","주간인구수(09~18)", "일최대이동인구수", "동일자치구행정동간이동인구수", "자치구간이동인구수"]]
group_df

,기준일ID,시군구명,주간인구수(09~18),일최대이동인구수,동일자치구행정동간이동인구수,자치구간이동인구수
0,20250327,서울시,1.127657e+07,5.964684e+06,1.829103e+06,2.516397e+06
1,20250327,종로구,4.301549e+05,2.755915e+05,2.639568e+04,1.696416e+05
2,20250327,중구,4.825689e+05,3.356017e+05,2.467516e+04,2.097482e+05
3,20250327,용산구,3.253080e+05,1.854691e+05,3.587798e+04,9.285225e+04
4,20250327,성동구,3.887017e+05,2.131262e+05,5.009943e+04,1.126245e+05
...,...,...,...,...,...,...
65905,20180405,관악구,4.662984e+05,2.526015e+05,1.443650e+05,7.471245e+04
65906,20180405,서초구,7.620019e+05,5.133245e+05,9.886215e+04,2.549145e+05
65907,20180405,강남구,1.105394e+06,7.517665e+05,1.402224e+05,3.854011e+05
65908,20180405,송파구,7.703412e+05,4.373945e+05,1.853804e+05,1.379524e+05


In [7]:
group_df = group_df.astype({"주간인구수(09~18)": int, "일최대이동인구수": int, "동일자치구행정동간이동인구수":int, "자치구간이동인구수":int})

In [8]:
weekends_2024 = [
   20240106, 20240107, 20240113, 20240114, 20240120, 20240121, 20240127,
   20240128, 20240203, 20240204, 20240210, 20240211, 20240217, 20240218,
   20240224, 20240225, 20240302, 20240303, 20240309, 20240310, 20240316,
   20240317, 20240323, 20240324, 20240330, 20240331, 20240406, 20240407,
   20240413, 20240414, 20240420, 20240421, 20240427, 20240428, 20240504,
   20240505, 20240511, 20240512, 20240518, 20240519, 20240525, 20240526,
   20240601, 20240602, 20240608, 20240609, 20240615, 20240616, 20240622,
   20240623, 20240629, 20240630, 20240706, 20240707, 20240713, 20240714,
   20240720, 20240721, 20240727, 20240728, 20240803, 20240804, 20240810,
   20240811, 20240817, 20240818, 20240824, 20240825, 20240831, 20240901,
   20240907, 20240908, 20240914, 20240915, 20240921, 20240922, 20240928,
   20240929, 20241005, 20241006, 20241012, 20241013, 20241019, 20241020,
   20241026, 20241027, 20241102, 20241103, 20241109, 20241110, 20241116,
   20241117, 20241123, 20241124, 20241130, 20241201, 20241207, 20241208,
   20241214, 20241215, 20241221, 20241222, 20241228, 20241229
]

In [19]:
select_sql("select count(day) from population")

[(2704,)]

In [9]:
keep_list = []
for index ,day in group_df.iterrows():
    if day["기준일ID"] in weekends_2024:
        keep_list.append(index)

weekend_df= group_df.loc[keep_list]
weekend_df

,기준일ID,시군구명,주간인구수(09~18),일최대이동인구수,동일자치구행정동간이동인구수,자치구간이동인구수
2262,20241229,서울시,10400560,4302447,1628558,1489784
2263,20241229,종로구,304280,176442,23231,93768
2264,20241229,중구,294836,163741,21412,90022
2265,20241229,용산구,306863,150266,31886,67448
2266,20241229,성동구,335385,139841,44689,61238
...,...,...,...,...,...,...
11591,20240106,관악구,466564,166346,82063,48197
11592,20240106,서초구,593280,291198,65000,122711
11593,20240106,강남구,871981,439294,108192,183216
11594,20240106,송파구,790954,333489,128833,90142


In [ ]:
execute_sql("""d""")

In [ ]:
[('id', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('day', 'datetime', 'NO', '', None, ''),
 ('region', 'varchar(10)', 'YES', '', None, ''),
 ('Daytime_person', 'int', 'YES', '', None, ''),
 ('move_person', 'int', 'YES', '', None, ''),
 ('move_dong', 'int', 'YES', '', None, ''),
 ('move_gu', 'int', 'YES', '', None, '')]

In [15]:
from datetime import datetime

sql = "insert into population (day, region, daytime_person, move_person, move_dong, move_gu) values( %s, %s, %s, %s, %s,%s)"
columns = ['기준일ID',"시군구명","주간인구수(09~18)", "일최대이동인구수","동일자치구행정동간이동인구수", "자치구간이동인구수"]



for index, row in weekend_df.iterrows():
    # 날짜 문자열을 datetime 객체로 변환 (필요한 경우)
    date_str = str(row['기준일ID'])
    date_obj = datetime.strptime(date_str, '%Y%m%d')
    print(row)
    params = (date_obj, row['시군구명'], row['주간인구수(09~18)'], row['일최대이동인구수'], row['동일자치구행정동간이동인구수'], row['자치구간이동인구수'])
    execute_sql(sql, params)

기준일ID             20241229
시군구명                   서울시
주간인구수(09~18)      10400560
일최대이동인구수           4302447
동일자치구행정동간이동인구수     1628558
자치구간이동인구수          1489784
Name: 2262, dtype: object
기준일ID             20241229
시군구명                   종로구
주간인구수(09~18)        304280
일최대이동인구수            176442
동일자치구행정동간이동인구수       23231
자치구간이동인구수            93768
Name: 2263, dtype: object
기준일ID             20241229
시군구명                    중구
주간인구수(09~18)        294836
일최대이동인구수            163741
동일자치구행정동간이동인구수       21412
자치구간이동인구수            90022
Name: 2264, dtype: object
기준일ID             20241229
시군구명                   용산구
주간인구수(09~18)        306863
일최대이동인구수            150266
동일자치구행정동간이동인구수       31886
자치구간이동인구수            67448
Name: 2265, dtype: object
기준일ID             20241229
시군구명                   성동구
주간인구수(09~18)        335385
일최대이동인구수            139841
동일자치구행정동간이동인구수       44689
자치구간이동인구수            61238
Name: 2266, dtype: object
기준일ID             20241229
시군구명                   광진구
주간인구수(

In [21]:
sater_list = [20240106, 20240113, 20240120, 20240127, 20240203, 20240210, 20240217, 20240224, 20240302, 20240309, 20240316, 20240323, 20240330, 20240406, 20240413, 20240420, 20240427, 20240504, 20240511, 20240518, 20240525, 20240601, 20240608, 20240615, 20240622, 20240629, 20240706, 20240713, 20240720, 20240727, 20240803, 20240810, 20240817, 20240824, 20240831, 20240907, 20240914, 20240921, 20240928, 20241005, 20241012, 20241019, 20241026, 20241102, 20241109, 20241116, 20241123, 20241130, 20241207, 20241214, 20241221, 20241228]

In [25]:
sundays_2024 = [
    20240107, 20240114, 20240121, 20240128, 
    20240204, 20240211, 20240218, 20240225, 
    20240303, 20240310, 20240317, 20240324, 20240331, 
    20240407, 20240414, 20240421, 20240428, 
    20240505, 20240512, 20240519, 20240526, 
    20240602, 20240609, 20240616, 20240623, 20240630, 
    20240707, 20240714, 20240721, 20240728, 
    20240804, 20240811, 20240818, 20240825, 
    20240901, 20240908, 20240915, 20240922, 20240929, 
    20241006, 20241013, 20241020, 20241027, 
    20241103, 20241110, 20241117, 20241124, 
    20241201, 20241208, 20241215, 20241222, 20241229
]

In [26]:
keep_list = []
for index ,day in weekend_df.iterrows():
    if day["시군구명"] == "서울시":
        if day['기준일ID'] in sater_list:
            keep_list.append(index)

sater_df= weekend_df.loc[keep_list]
sater_df

,기준일ID,시군구명,주간인구수(09~18),일최대이동인구수
2288,20241228,서울시,10748355,4752680
2470,20241221,서울시,10933598,5007468
2652,20241214,서울시,11071881,5294159
2834,20241207,서울시,10940564,5109842
3016,20241130,서울시,10823314,5011048
3198,20241123,서울시,10710154,5065557
3380,20241116,서울시,10567624,5007884
3562,20241109,서울시,10613685,5098482
3744,20241102,서울시,10612456,5111935
3926,20241026,서울시,10588215,4883892


In [27]:
keep_list = []
for index ,day in weekend_df.iterrows():
    if day["시군구명"] == "서울시":
        if day['기준일ID'] in sundays_2024:
            keep_list.append(index)

sunday_df= weekend_df.loc[keep_list]
sunday_df

,기준일ID,시군구명,주간인구수(09~18),일최대이동인구수
2262,20241229,서울시,10400560,4302447
2444,20241222,서울시,10729549,4574007
2626,20241215,서울시,10759150,4609254
2808,20241208,서울시,10728148,4553399
2990,20241201,서울시,10732841,4664749
3172,20241124,서울시,10636483,4735278
3354,20241117,서울시,10548317,4698909
3536,20241110,서울시,10474960,4694776
3718,20241103,서울시,10493974,4731885
3900,20241027,서울시,10586958,4630259


In [44]:
execute_sql("truncate table population")

[]

In [ ]:
sql = "insert into population (day, region, Daytime_person, move_person) values( %s, %s, %s, %s)"
columns = ['기준일ID', "시군구명", "주간인구수(09~18)", "일최대이동인구수"]

for index, row in weekend_df.iterrows():
    params = tuple(row[columns])
    execute_sql(sql, params)
    print(row)

기준일ID           20241229
시군구명                 서울시
주간인구수(09~18)    10400560
일최대이동인구수         4302447
Name: 2262, dtype: object
기준일ID           20241229
시군구명                 종로구
주간인구수(09~18)      304280
일최대이동인구수          176442
Name: 2263, dtype: object
기준일ID           20241229
시군구명                  중구
주간인구수(09~18)      294836
일최대이동인구수          163741
Name: 2264, dtype: object
기준일ID           20241229
시군구명                 용산구
주간인구수(09~18)      306863
일최대이동인구수          150266
Name: 2265, dtype: object
기준일ID           20241229
시군구명                 성동구
주간인구수(09~18)      335385
일최대이동인구수          139841
Name: 2266, dtype: object
기준일ID           20241229
시군구명                 광진구
주간인구수(09~18)      367641
일최대이동인구수          141620
Name: 2267, dtype: object
기준일ID           20241229
시군구명                동대문구
주간인구수(09~18)      342026
일최대이동인구수          133319
Name: 2268, dtype: object
기준일ID           20241229
시군구명                 중랑구
주간인구수(09~18)      337534
일최대이동인구수          121118
Name: 2269, dtype:

KeyboardInterrupt: 

In [29]:
execute_sql("select * from population")

[]

In [18]:
sater_df.reset_index(drop=True, inplace=True)  
sater_df

,기준일ID,일최대이동인구수
0,20241228,4752680
1,20241221,5007468
2,20241214,5294159
3,20241207,5109842
4,20241130,5011048
5,20241123,5065557
6,20241116,5007884
7,20241109,5098482
8,20241102,5111935
9,20241026,4883892


In [31]:
sater_df = sater_df = sater_df.astype(int)

In [32]:
sater_df

,기준일ID,일최대이동인구수
0,20241228,4752680
1,20241221,5007468
2,20241214,5294159
3,20241207,5109842
4,20241130,5011048
5,20241123,5065557
6,20241116,5007884
7,20241109,5098482
8,20241102,5111935
9,20241026,4883892


### 일 최대 이동인구수가 많은 10개 날

In [57]:
sater_max = sater_df.sort_values(by = "일최대이동인구수", ascending=False).head(26)

In [38]:
sater_min = sater_df.sort_values(by = "일최대이동인구수", ascending=True).head(10)

In [58]:
sater_max.reset_index(drop=True, inplace=True)
sater_max

,기준일ID,일최대이동인구수
0,20241214,5294159
1,20240406,5155401
2,20241102,5111935
3,20241207,5109842
4,20241109,5098482
5,20241123,5065557
6,20241130,5011048
7,20241116,5007884
8,20241221,5007468
9,20240427,4960936


In [40]:
sater_min.reset_index(drop=True, inplace=True)
sater_min

,기준일ID,일최대이동인구수
0,20240210,4046702
1,20240127,4337415
2,20240914,4350010
3,20240727,4452400
4,20240803,4476579
5,20240302,4483383
6,20240817,4529982
7,20240720,4533281
8,20240608,4553064
9,20240810,4606317


In [41]:
sater_max["기준일ID"]

0    20241214
1    20240406
2    20241102
3    20241207
4    20241109
5    20241123
6    20241130
7    20241116
8    20241221
9    20240427
Name: 기준일ID, dtype: int64

In [42]:
sater_min["기준일ID"]

0    20240210
1    20240127
2    20240914
3    20240727
4    20240803
5    20240302
6    20240817
7    20240720
8    20240608
9    20240810
Name: 기준일ID, dtype: int64

In [46]:
sater_avg = int(sater_df['일최대이동인구수'].mean())
sater_avg

4777974

### 토요일과 일요일 사이의 차가 극명하다 

--------------------

### 토요일끼리 비교해보자

In [4]:
sater_list = [20240106, 20240113, 20240120, 20240127, 20240203, 20240210, 20240217, 20240224, 20240302, 20240309, 20240316, 20240323, 20240330, 20240406, 20240413, 20240420, 20240427, 20240504, 20240511, 20240518, 20240525, 20240601, 20240608, 20240615, 20240622, 20240629, 20240706, 20240713, 20240720, 20240727, 20240803, 20240810, 20240817, 20240824, 20240831, 20240907, 20240914, 20240921, 20240928, 20241005, 20241012, 20241019, 20241026, 20241102, 20241109, 20241116, 20241123, 20241130, 20241207, 20241214, 20241221, 20241228]

In [12]:
keep_list = []
for index ,day in group_df.iterrows():
    if day["기준일ID"] in sater_list:
        keep_list.append(index)

sater_df= group_df.loc[keep_list]
sater_df

,기준일ID,주간인구수(09~18),야간인구수(19~08),일최대이동인구수
2289,20241228,364915.2529,249154.1527,261683.6260
2471,20241221,373712.6522,250492.3650,268877.2194
2653,20241214,338507.8404,247373.8529,211773.0148
2835,20241207,329971.5711,245835.2289,203842.0333
3017,20241130,335569.0662,245438.9395,211270.2591
3199,20241123,357765.0660,247332.9834,233858.1546
3381,20241116,367595.2104,246668.9909,247256.7639
3563,20241109,374736.9108,248170.5269,261707.9794
3745,20241102,374703.9382,248579.6557,261617.8455
3927,20241026,368641.6214,249177.6558,240659.6114


In [14]:
sater_df.reset_index(drop= [0], inplace=False)

,기준일ID,주간인구수(09~18),야간인구수(19~08),일최대이동인구수
0,20241228,364915.2529,249154.1527,261683.6260
1,20241221,373712.6522,250492.3650,268877.2194
2,20241214,338507.8404,247373.8529,211773.0148
3,20241207,329971.5711,245835.2289,203842.0333
4,20241130,335569.0662,245438.9395,211270.2591
5,20241123,357765.0660,247332.9834,233858.1546
6,20241116,367595.2104,246668.9909,247256.7639
7,20241109,374736.9108,248170.5269,261707.9794
8,20241102,374703.9382,248579.6557,261617.8455
9,20241026,368641.6214,249177.6558,240659.6114


### 일 최대 이동인구수에 대해 알아보자 

----------------------